In [2]:
import pandas as pd
import numpy as np

TRADES_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-trades_regression_insp500_spyfilter_cap15.parquet"
EQUITY_FILE = "./13-trading_output_regression_insp500_spyfilter_cap15/13-equity_curve_regression_insp500_spyfilter_cap15.parquet"

REPORT_DAY = "Wednesday"
WEEKLY_WITHDRAWAL = 3000.0

# ------------------------------------------------------------
# LOAD DATA
# ------------------------------------------------------------

trades = pd.read_parquet(TRADES_FILE)
trades["exec_date"] = pd.to_datetime(trades["exec_date"])
trades = trades.sort_values("exec_date")

eq = pd.read_parquet(EQUITY_FILE)
eq["date"] = pd.to_datetime(eq["date"])
eq = eq.sort_values("date").set_index("date")

trades_by_date = {d: g for d, g in trades.groupby("exec_date")}

# ------------------------------------------------------------
# WEEKLY DATES (SOURCE OF TRUTH = EQUITY CURVE)
# ------------------------------------------------------------

weekly_dates = [d for d in eq.index if d.day_name() == REPORT_DAY]

records = []
cumulative_withdrawals = 0.0
ever_negative = False

for dt in weekly_dates:

    # -------------------------
    # CASH BEFORE WITHDRAWAL
    # -------------------------
    if dt in trades_by_date:
        day_trades = trades_by_date[dt]
        cash_before = float(day_trades["cash_after"].iloc[-1])
    else:
        cash_before = float(eq.loc[dt, "cash"])

    # -------------------------
    # APPLY WEEKLY WITHDRAWAL
    # -------------------------
    cash_after = cash_before - WEEKLY_WITHDRAWAL
    cumulative_withdrawals += WEEKLY_WITHDRAWAL

    negative_flag = cash_after < 0
    ever_negative = ever_negative or negative_flag

    # -------------------------
    # PORTFOLIO VALUES (UNCHANGED)
    # -------------------------
    portfolio_value = float(eq.loc[dt, "portfolio_value"])
    securities_value = portfolio_value - cash_before  # pre-withdrawal view

    records.append({
        "date": dt,
        "portfolio_value": portfolio_value,
        "cash_before_withdrawal": cash_before,
        "weekly_withdrawal": WEEKLY_WITHDRAWAL,
        "cash_after_withdrawal": cash_after,
        "securities_value": securities_value,
        "num_positions": int(eq.loc[dt, "num_positions"]),
        "negative_cash": negative_flag,
        "cumulative_withdrawals": cumulative_withdrawals
    })

# ------------------------------------------------------------
# OUTPUT
# ------------------------------------------------------------

weekly_df = pd.DataFrame(records).set_index("date")

print("\n=== WEEKLY CASH WITHDRAWAL STRESS TEST ===\n")
for dt, r in weekly_df.iterrows():
    flag = " **NEGATIVE CASH**" if r["negative_cash"] else ""
    print(
        f"{dt.date()} | "
        f"Portfolio ${r['portfolio_value']:,.2f} | "
        f"Cash Before ${r['cash_before_withdrawal']:,.2f} | "
        f"Cash After ${r['cash_after_withdrawal']:,.2f} | "
        f"Withdrawn ${r['weekly_withdrawal']:,.0f}{flag}"
    )

print("\nSUMMARY")
print("-" * 60)
print(f"Total weeks simulated: {len(weekly_df)}")
print(f"Cumulative withdrawals: ${cumulative_withdrawals:,.2f}")
print(f"Ever negative cash: {'YES' if ever_negative else 'NO'}")

weekly_df.to_csv("weekly_cash_withdrawal_stress_test.csv")
print("\nSaved weekly_cash_withdrawal_stress_test.csv")



=== WEEKLY CASH WITHDRAWAL STRESS TEST ===

1999-01-06 | Portfolio $360,000.00 | Cash Before $360,000.00 | Cash After $357,000.00 | Withdrawn $3,000
1999-01-13 | Portfolio $360,986.05 | Cash Before $162,354.19 | Cash After $159,354.19 | Withdrawn $3,000
1999-01-20 | Portfolio $372,676.52 | Cash Before $189,842.30 | Cash After $186,842.30 | Withdrawn $3,000
1999-01-27 | Portfolio $366,969.79 | Cash Before $168,476.40 | Cash After $165,476.40 | Withdrawn $3,000
1999-02-03 | Portfolio $379,673.49 | Cash Before $146,109.18 | Cash After $143,109.18 | Withdrawn $3,000
1999-02-10 | Portfolio $366,645.61 | Cash Before $197,174.61 | Cash After $194,174.61 | Withdrawn $3,000
1999-02-17 | Portfolio $368,537.82 | Cash Before $214,439.98 | Cash After $211,439.98 | Withdrawn $3,000
1999-02-24 | Portfolio $379,516.93 | Cash Before $212,084.92 | Cash After $209,084.92 | Withdrawn $3,000
1999-03-03 | Portfolio $375,279.92 | Cash Before $233,318.64 | Cash After $230,318.64 | Withdrawn $3,000
1999-03-10